# Нейронные сети. Архитектуры нейронных сетей

## Подготовка данных

В этом практическом заданий мы будем решать задачу классификации цифр на датасете `mnist` с помощью полносвязной и сверточной нейронной сети. Для этого мы будем использовать надстройку над `tensorflow`, которая называется `keras`. Для начала обсудим данные. `mnist` датасет состоит из черно-белых изображений цифр размером $28 \times 28$ пикселей. В данном случае, мы работаем с одним каналом, хотя в случае цветных изображений, общее число каналов равно трем. Загрузим наши данные используя функцию `load_data` объекта `mnist` из модуля `keras.dataset`. Перед выполнением этого задания убедитесь, что ваша версия `tensorflow` >= 1.4.

In [168]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Нормализуйте заруженные данные `x_train` и `x_test`. Для этого следует разделить числовое значение каждого пикселя на $255$. Далее, переведите `y_train` и `y_test` в one-hot представление, используя функцию `tf.keras.utils.to_categorical`. Наше первое задание будет заключатся в реализации полносвязной нейронной сети. Поэтому измените размерность тренировочных и тестовых данных с помощью метода `reshape`.
    >> np_vector.shape
    >> (28, 28)
    >> np_vector = np_vector.reshape(28 * 28)
    >> np_vector.shape
    >> (784,)

### *РЕШЕНИЕ*

In [169]:
num_classes = 10

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

y_train.shape

(60000, 10)

## Полносвязная нейронная сеть.

В этой части задания вам предлагается реализовать обычную нейронную сеть с использованием последовательной модели `tf.keras.models.Sequential`. Данная модель позволяет добавлять слои при помощи функции встроенной `add`. Наша нейронная сеть будет состоять всего лишь из одного скрытого слоя с количеством нейроннов равным $256$, функцией активации ReLU и с `input_shape=(784,)`, что соответствует количеству нейронов во входном слое нашей нейронной сети. Количество нейроннов в выходном слое равно количеству классов, в качестве функции активации нужно использовать softmax. Не забудьте вызвать `model.compile` после добавления слоев. Используйте в качестве функции потерь `categorical_crossentropy`, оптимизатор `adadelta` и метрику `accuracy`.

### *РЕШЕНИЕ*

In [170]:
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(784, )))
nn.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
       
nn.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

После этого, создайте модель и загрузите веса нейронной сети из файла `neural_networks.h5`. Какое количество настраиваемых параметров содержится в этой нейронной сети. Запишите это число в качестве первого ответа `answer1` на это задание. Оцените качетво на тестовой выборке и запишите это значение с точностью до трех знаков после запятой в переменную `answer2`.

In [171]:
nn.load_weights('neural_networks.h5')

In [172]:
answer1 = nn.count_params()

In [173]:
loss, accuracy = nn.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.06618442386388779
Test accuracy: 0.9819999933242798


In [174]:
answer2 = round(accuracy, 3)

### *РЕШЕНИЕ*

In [175]:
nn.get_config()

{'name': 'sequential_18',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 784),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_25_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_25',
    'trainable': True,
    'batch_input_shape': (None, 784),
    'dtype': 'float32',
    'units': 256,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_26',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},


## Сверточная нейронная сеть

Далее, вам предлагается реализовать сверточную нейронную сеть. 

* Размерность входного слоя $(28, 28, 1)$.
* Сверточный слой с $32$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Сверточный слой с $64$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Понижение размерности признаков.
* Полносвязный слой с 64 нейронами
* Выходной слой с количеством нейронов равному количеству классов.

Для этого предлагается использовать следующие классы `Convolution2D`, `MaxPooling2D` и `Flatten` для понижения размерности признаков. Все эти классы как и слой полносвязной нейронной сети `Dense` находятся в `tf.keras.layers`. Используйте ReLU в качестве функции активации во всех слоях, где это потребуется, кроме выходного, в нем по аналогии с прошлым заданием используется softmax. Аналогичная ситуация с функцией `compile` после добавления слоев.

### *РЕШЕНИЕ*

In [176]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

Теперь оцените качество получившейся модели на тестовой выборке. Для этого измените размерность `x_train` и `x_test` на размерность входного слоя. Загрузите веса `conv_net.h5`. Запишите количество параметров этой сверточной нейронной сети в `answer3`. Сравните его с количеством параметром в полносвязной нейронной сети, которую мы реализовали ранее. Оценку качества запишите в `answer4` с точностью до 3 трех знаков после запятой.

In [177]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [178]:
answer3 = cnn.count_params()

### *РЕШЕНИЕ*

In [179]:
cnn.load_weights('conv_net.h5')

In [180]:
loss, accuracy = cnn.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.030455416068434715
Test accuracy: 0.992900013923645


In [181]:
answer4 = round(accuracy, 3)

# Строка с ответами

In [182]:
output = "nn params {0}\n nn score {1:.3f}\ncnn params {2}\ncnn score {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))

nn params 203530
 nn score 0.982
cnn params 121930
cnn score 0.993
